# Constant Expressions

A **constant expression** is an expression that can be evaluated at compile time.
* Within C++03, such an expression can only involve litteral integral values and variables which are themselves initialized with constant expressions.
* C++11 allows floating point numbers, simplistic functions and all "litteral" types, including your own classes.
* C++14 open widely what is allowed in a `constexpr` function.
* C++17 adds `if constexpr`, also evaluated at compile time, making obsolete some old meta-programming tips.

**Moving some evaluations at the compile-time obsviously accelerates runtime execution**, but it also avoid some usual pitfalls:
* uncertain order of global variable initialization when launching an executable,
* uncertain order of execution of multiple threads.
This proves especially important for class such as `std::mutex` or `std::atomic`.

## `constexpr` variables

A `constexpr` variable is implicitly `const`, and **must** be evaluated at compile time. Unlike `const`, it will not fill some memory slot, and its value will be directly substituted wherever it is used in the code. Unlike `#define`, it is evaluated only once, and will not suffer the usual preprocessor pitfalls.

Also, floating numbers are allowed, and one can initialised within a class definition any member variable which is both `static` et `constexpr`. For example, this would not compile in ancient C++:

In [ ]:
struct X
 {
  static constexpr float val = 3.14 ;
 } ;

##  `constexpr` functions

A call to a `constexpr` function **may possibly** be evaluated at compile time, if the arguments to the call are themselves `constant expressions`. For example, one can use `constexpr` functions when computing the size of some fixed array.

In this example, would you find out which calls to `square()` are evaluated at compile time, and which ones are evaluated at run time ?
<!--
* t1 : compile time, constexpr.
* t2 : compile time, constant expression.
* t3 : run time, the constance is lst once stored in a non-const varibale ?
-->

In [ ]:
constexpr int square( int n )
 { return n*n ; }

In [ ]:
#include <iostream>
{
  constexpr int t1 = 2 ;
  const int t2 = 3 ;
  int t3 = 4 ;
    
  std::cout << square(t1) << std::endl ;
  std::cout << square(t2) << std::endl ;
  std::cout << square(t3) << std::endl ;
}

## `if constexpr` (C++17)

This kind of `if` is evaluated at compile time. Of course, the condition must be a constant expression. This new feature is largely better than the preprocessor `#if`, because its condition can be evaluated by the compiler.

In this first example, we compare two numbers, with a margin for floating point numbers.

In [ ]:
#include <cmath>

template <class T>
constexpr auto precision_threshold = T(0.001) ;

template <class T>
constexpr bool close_enough(T a, T b)
 {
  if constexpr (std::is_floating_point_v<T>)
    return std::abs(a-b) < precision_threshold<T> ;
  else
    return a == b ;
 } 

In [ ]:
#include <iostream>
{
  std::cout<<close_enough(11,12)<<std::endl ;
  std::cout<<close_enough(12,12)<<std::endl ;
  std::cout<<close_enough(3.14,3.15)<<std::endl ;
  std::cout<<close_enough(3.1416,3.1417)<<std::endl ;
}

Below, the implementation of `my_advance()` is optimized for random-access iterators. Why would the compilation fail without `constexpr`, if I use `advance()` with a `std::list`iterator ?

In [ ]:
#include <iterator>

template <typename Iterator, typename Dist>
void my_advance( Iterator & i, Dist n )
 {
  using cat = typename std::iterator_traits<Iterator>::iterator_category ;
  if constexpr (std::is_same_v<cat,std::random_access_iterator_tag>)
   { i += n ; }
  else
   {
    if (n >= 0) while (n--) ++i ;
    else while (n++) --i ;
   }
 }

## `constexpr` classes & objects

Succintly speaking, a class is allowed in constant expressions, provided its member variables are also allowed, and there is a `constexpr` constructor which explicitly initialize each of them:

In [ ]:
class Point2d
 {
  public :
    constexpr Point2d( float x, float y ) : m_x(x), m_y(y) {}
  private :
    float m_x, m_y ;
 } ;

In [ ]:
constexpr Point2d origine(0.0,0.0) ;

Actually, the class must comply with all the requirements of the "Literal Type" category, which we will not detail here. Predefined numeric types are compliants. `std::string` is not.

## `constexpr` template functions

For what concerns a template function, the `constexpr` keyword is taken into account if and only if the type parameters are compatible with a compile-time evaluation. If not, it is simply ignored. This flexibility avoid painful multiple definitions of a given function.

In [ ]:
template<typename T>
constexpr T sum(T a,T b)
{ return a+b ; }

In [ ]:
#include <string>
{
    constexpr int i = sum(3,42) ; // OK, sum<int> is constexpr
    std::string s = sum(std::string("hello"),std::string(" world")); // OK, but sum<std::string> isn’t constexpr
}

## Exercise

Correct the code below, so that the compiler accepts that the instance of `FromTo`, at the beginning of `count`, is evaluable at compile time, as requested by `constexpr`.

In [ ]:
%%file tmp.constexpr.cpp

#include <iostream>
#include <string>

class FromTo
 {
  public :
    struct Itr
     {
      int val ;
      bool operator!=( Itr const & itr ) const { return (val!=itr.val) ;  }
      int operator*() const { return val ; }
      Itr & operator++() { ++val ; return *this ; }
     } ;
    FromTo( int from, int to ) : m_from{from}, m_to{to} {}
    Itr begin() const { return Itr{m_from} ; }
    Itr end() const { return Itr{m_to} ; }
  private :
    int const m_from, m_to ;
 } ;

void count( std::string const & name, int from, int to )
 {
  constexpr FromTo range(from,to) ;
  std::cout<<name ;
  for ( auto rank : range )
   { std::cout<<", "<<rank ; }
  std::cout<<std::endl ;
 }

int main()
 {
  count("First",0,3) ;
  count("Second",0,7) ;
  return 0 ;
 }

In [ ]:
!rm -f tmp.constexpr.exe && g++ -std=c++17 tmp.constexpr.cpp -o tmp.constexpr.exe

In [ ]:
!./tmp.constexpr.exe

## Ressources
* <http://en.cppreference.com/w/cpp/language/constant_expression>
* <http://en.cppreference.com/w/cpp/concept/LiteralType>
* <http://www.cprogramming.com/c++11/c++11-compile-time-processing-with-constexpr.html> * http://meetingcpp.com/blog/items/How-if-constexpr-simplifies-your-code-in-Cpp17.html
* https://www.codingame.com/playgrounds/2205/7-features-of-c17-that-will-simplify-your-code/constexpr-if
* https://linuxfr.org/news/cpp17-branche-a-la-compilation-if-constexpr
* https://solarianprogrammer.com/2017/12/27/cpp-17-constexpr-everything-as-much-as-the-compiler-can/


© *CNRS 2020*  
*Assembled and written by David Chamont, this work is made available according to the terms of the*  
[*Creative Commons License - Attribution - NonCommercial - ShareAlike 4.0 International*](http://creativecommons.org/licenses/by-nc-sa/4.0/)